# 6 - 7. Mô tả và đánh giá dữ liệu & Tiền xử lí dữ liệu

* Load toàn bộ review vào một dataframe duy nhất.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import modules.utils as Utils
import modules.processor as Processor
import numpy as np
import pandas as pd

In [3]:
# Lấy tất cả các directory path của các lần ta tiến hành crawl data
dir_paths = Utils.getAllFolderPath("./data/product_reviews/")

dir_paths

['./data/product_reviews/product_reviews_01/',
 './data/product_reviews/product_reviews_02/',
 './data/product_reviews/product_reviews_03/',
 './data/product_reviews/product_reviews_00/']

In [4]:
# Đọc toàn bộ các review từ các file csv
reviews = Utils.readReviews(dir_paths)

In [5]:
reviews.head()

,raw_comment,rating
0,"Mình mua size L. Cổ tay siêu bé, như size S ấy...",1
1,"Size S M L XL\n\nForm áo cực kì dễ mang, thiết...",5
2,Áo khá đẹp vừa với dáng giao hàng cực kì nhanh...,5
3,Đẹp rất hài lòng okokokokokokokokokokokokoko...,5
4,"Đẹp, ôm dáng, mặc đẹp lắm mà form nhỏ. Mình 60...",5


In [6]:
print("Tập dữ liệu có {} bình luận.".format(reviews.shape[0]))

Tập dữ liệu có 278159 bình luận.


* Đếm tần số xuất hiện của từng rating.

In [7]:
reviews['rating'].value_counts()

5    260555
4      9646
3      4194
1      2308
2      1456
Name: rating, dtype: int64

> **Nhận xét**:
> * Nhìn chung tuy ta crawl được hơn 200,000 quan sát nhưng có sự chênh lệch lớn giữa các rating.
> * Nhìn qua ta thấy đa phần là các rating được đánh giá 5 sao, điều này cũng dễ hiểu vì hệ thống recommend của Shopee sẽ ưu tiên gợi ý cho khách hàng những sản phẩm có đánh giá tốt. Và sẽ hạn chế hoặc thậm chí là ko gợi ý các mặc hàng bị đánh giá kém. Nên với địa vị là người đi trộm dữ liệu như chúng ta thì ko có cách nào khắc phục điều này.
> * Bây giờ, do ta cần chi ra hai lớp là negative và positive nên những comment mà `rating` $\geq 4$ sẽ được cho vào nhóm positive, ngược lại là nhóm negative. 

* Tiến hành label cho `reviews` với các giá `rating` $< 4$ sẽ thuộc nhóm negative còn lại là nhóm positive.

In [8]:
reviews = Utils.labelRating(reviews)

In [9]:
reviews.head()

,raw_comment,rating,label
0,"Mình mua size L. Cổ tay siêu bé, như size S ấy...",1,0
1,"Size S M L XL\n\nForm áo cực kì dễ mang, thiết...",5,1
2,Áo khá đẹp vừa với dáng giao hàng cực kì nhanh...,5,1
3,Đẹp rất hài lòng okokokokokokokokokokokokoko...,5,1
4,"Đẹp, ôm dáng, mặc đẹp lắm mà form nhỏ. Mình 60...",5,1


* Bây giờ ta sẽ xóa đi feature `rating` vì về sau ta sẽ ko cần dúng đến nó nữa, và tiến hành đếm số lượng quan sát của từng nhóm trên `label`

In [10]:
reviews = reviews.drop(columns=['rating'])

Processor.printAfterProcess(reviews)
reviews.head()

Shape: (278159, 2)
1    270201
0      7958
Name: label, dtype: int64


,raw_comment,label
0,"Mình mua size L. Cổ tay siêu bé, như size S ấy...",0
1,"Size S M L XL\n\nForm áo cực kì dễ mang, thiết...",1
2,Áo khá đẹp vừa với dáng giao hàng cực kì nhanh...,1
3,Đẹp rất hài lòng okokokokokokokokokokokokoko...,1
4,"Đẹp, ôm dáng, mặc đẹp lắm mà form nhỏ. Mình 60...",1


* Ghi `reviews` ra file csv

In [11]:
reviews.to_csv("./data/reviews.csv", index=False)

<hr>

* Một trong những vấn đề đầu tiên và tối quan trong khi xử lí với dữ liệu văn bản là kiểm tra xem liệu text `a` có cùng cách biểu diễn với text `b` hay không.
* Một ví dụ dễ hiểu là giả sử ta có biến `a = 'đẹp'` và biến `b = 'đep'`, nhưng khi ta compare hai biến này `a == b` thì kết quả sẽ ra False, nguyên nhân là do chúng sử dụng mã hóa unicode khác nhau, có thể `a` dùng unicode-8 và `b` dùng unicode-16.
* Vậy điều đầu tiên ta cần làm là phải đưa tất cả các text về cùng một **chuẩn** duy nhất, ta có thể làm điều này bằng cách sử dụng `unicodedata.normalize()` từ package chuẩn `unicodedata` của Python. _(tham khảo thêm tại đây [https://www.kite.com/python/docs/unicodedata.normalize](https://www.kite.com/python/docs/unicodedata.normalize))_
* Dưới đây là ví dụ cho trường hợp này:

In [12]:
import unicodedata

a = 'đẹp, rất hài lòng'
b = 'đẹp, rất hài lòng'

a == b

False

In [13]:
''' Sử dụng chuẩn NFD '''
a = unicodedata.normalize('NFD', a)
b = unicodedata.normalize('NFD', b)

print("String a: {}, Type a: {}".format(a, type(a)))
print("String b: {}, Type b: {}".format(b, type(b)))

String a: đẹp, rất hài lòng, Type a: <class 'str'>
String b: đẹp, rất hài lòng, Type b: <class 'str'>


In [14]:
a == b

True

* Bây giờ, ta sẽ tạo một feature có tên là `normalize_comment`, trải qua 2 bước:
  * `lower()` cho text.
  * Chuẩn hóa bằng `unicodedata.normalize()`

In [15]:
reviews['normalize_comment'] = reviews['raw_comment'].apply(lambda cmt: Processor.normalizeComment(cmt))

reviews.head()

,raw_comment,label,normalize_comment
0,"Mình mua size L. Cổ tay siêu bé, như size S ấy...",0,"mình mua size l. cổ tay siêu bé, như siz..."
1,"Size S M L XL\n\nForm áo cực kì dễ mang, thiết...",1,"size s m l xl\n\nform áo cực kì dễ mang,..."
2,Áo khá đẹp vừa với dáng giao hàng cực kì nhanh...,1,áo khá đẹp vừa với dáng giao hàng cư...
3,Đẹp rất hài lòng okokokokokokokokokokokokoko...,1,đẹp rất hài lòng okokokokokokokokokokokok...
4,"Đẹp, ôm dáng, mặc đẹp lắm mà form nhỏ. Mình 60...",1,"đẹp, ôm dáng, mặc đẹp lắm mà form nho..."


In [16]:
reviews['normalize_comment'] = reviews['raw_comment'].apply(lambda cmt: Processor.normalizeComment(cmt))

* Ta cũng sẽ chuẩn hóa cho cột `raw_comment` nhưng không `lower()` chúng.

In [17]:
reviews['raw_comment'] = reviews['raw_comment'].apply(lambda cmt: Processor.normalizeComment(cmt, False))

reviews.head()

,raw_comment,label,normalize_comment
0,"Mình mua size L. Cổ tay siêu bé, như siz...",0,"mình mua size l. cổ tay siêu bé, như siz..."
1,"Size S M L XL\n\nForm áo cực kì dễ mang,...",1,"size s m l xl\n\nform áo cực kì dễ mang,..."
2,Áo khá đẹp vừa với dáng giao hàng cư...,1,áo khá đẹp vừa với dáng giao hàng cư...
3,Đẹp rất hài lòng okokokokokokokokokokokok...,1,đẹp rất hài lòng okokokokokokokokokokokok...
4,"Đẹp, ôm dáng, mặc đẹp lắm mà form nho...",1,"đẹp, ôm dáng, mặc đẹp lắm mà form nho..."


<hr>

* Chúng ta biết rằng, các comment của các sản phẩm đôi khi sẽ chứa các URL do người bán hàng chèn vào để giúp khách hàng có thể click vào để xem các mặt hàng khác, chúng là các noise sample mà ta cần phải loại bỏ khỏi dataset của chúng ta.
* Hình dưới đây là kết quả cho ra khi ta thử search cụm từ `http` thì nó cho ra hơn 600 mẫu dữ liệu chứa URL. Ta cần loại bỏ các mẫu này.<br>
  ![](./images/04.png)

In [18]:
reviews['contain_url'] = reviews['normalize_comment'].apply(lambda cmt: Processor.containsURL(cmt))

Processor.printAfterProcess(reviews, 'contain_url')
reviews.head()

Shape: (278159, 4)
0    277589
1       570
Name: contain_url, dtype: int64


,raw_comment,label,normalize_comment,contain_url
0,"Mình mua size L. Cổ tay siêu bé, như siz...",0,"mình mua size l. cổ tay siêu bé, như siz...",0
1,"Size S M L XL\n\nForm áo cực kì dễ mang,...",1,"size s m l xl\n\nform áo cực kì dễ mang,...",0
2,Áo khá đẹp vừa với dáng giao hàng cư...,1,áo khá đẹp vừa với dáng giao hàng cư...,0
3,Đẹp rất hài lòng okokokokokokokokokokokok...,1,đẹp rất hài lòng okokokokokokokokokokokok...,0
4,"Đẹp, ôm dáng, mặc đẹp lắm mà form nho...",1,"đẹp, ôm dáng, mặc đẹp lắm mà form nho...",0


* Bây giờ chúng ta chỉ sẽ lấy các comment mà không chứa URL

In [19]:
reviews = reviews[reviews['contain_url'] == 0]
reviews = reviews.drop(columns=['contain_url']).reset_index(drop=True) # xóa cột `contain_url`

Processor.printAfterProcess(reviews)
reviews.head()

Shape: (277589, 3)
1    269633
0      7956
Name: label, dtype: int64


,raw_comment,label,normalize_comment
0,"Mình mua size L. Cổ tay siêu bé, như siz...",0,"mình mua size l. cổ tay siêu bé, như siz..."
1,"Size S M L XL\n\nForm áo cực kì dễ mang,...",1,"size s m l xl\n\nform áo cực kì dễ mang,..."
2,Áo khá đẹp vừa với dáng giao hàng cư...,1,áo khá đẹp vừa với dáng giao hàng cư...
3,Đẹp rất hài lòng okokokokokokokokokokokok...,1,đẹp rất hài lòng okokokokokokokokokokokok...
4,"Đẹp, ôm dáng, mặc đẹp lắm mà form nho...",1,"đẹp, ôm dáng, mặc đẹp lắm mà form nho..."


> **Nhận xét**
> * Đa phần là các bình luận thuộc nhóm positive sẽ chứa các URL, cũng dễ hiểu vì họ quảng cáo mà 😅.

<hr>

* Tiếp theo, ta cũng cần xóa các comment mà chữ IN HOA chiếm quá 50% độ dài comment, các comment này khả năng cao cũng là quảng cáo, vì người bán họ muốn làm nổi bật bình luận này lên so với các bình luận còn lại.

In [20]:
reviews['contain_adv'] = reviews['raw_comment'].apply(lambda cmt: Processor.containAdvertisement(cmt))

Processor.printAfterProcess(reviews, 'contain_adv')
reviews.head()

Shape: (277589, 4)
0    270936
1      6653
Name: contain_adv, dtype: int64


,raw_comment,label,normalize_comment,contain_adv
0,"Mình mua size L. Cổ tay siêu bé, như siz...",0,"mình mua size l. cổ tay siêu bé, như siz...",0
1,"Size S M L XL\n\nForm áo cực kì dễ mang,...",1,"size s m l xl\n\nform áo cực kì dễ mang,...",0
2,Áo khá đẹp vừa với dáng giao hàng cư...,1,áo khá đẹp vừa với dáng giao hàng cư...,0
3,Đẹp rất hài lòng okokokokokokokokokokokok...,1,đẹp rất hài lòng okokokokokokokokokokokok...,0
4,"Đẹp, ôm dáng, mặc đẹp lắm mà form nho...",1,"đẹp, ôm dáng, mặc đẹp lắm mà form nho...",0


> **Nhận xét**
> * Các mẫu tìm năng chứa quảng cáo khá cao, lên đến hơn 3000 sample. Ta có thể xóa chúng.

In [21]:
reviews = reviews[reviews['contain_adv'] == 0]
reviews = reviews.drop(columns=['contain_adv']).reset_index(drop=True) # xóa cột `contain_adv`

Processor.printAfterProcess(reviews)
reviews.head()

Shape: (270936, 3)
1    263084
0      7852
Name: label, dtype: int64


,raw_comment,label,normalize_comment
0,"Mình mua size L. Cổ tay siêu bé, như siz...",0,"mình mua size l. cổ tay siêu bé, như siz..."
1,"Size S M L XL\n\nForm áo cực kì dễ mang,...",1,"size s m l xl\n\nform áo cực kì dễ mang,..."
2,Áo khá đẹp vừa với dáng giao hàng cư...,1,áo khá đẹp vừa với dáng giao hàng cư...
3,Đẹp rất hài lòng okokokokokokokokokokokok...,1,đẹp rất hài lòng okokokokokokokokokokokok...
4,"Đẹp, ôm dáng, mặc đẹp lắm mà form nho...",1,"đẹp, ôm dáng, mặc đẹp lắm mà form nho..."


> **Nhận xét**
> * Lại một lần nữa các comment có khả năng cao là quảng cáo này lại đa phần là thuộc nhóm positive.

<hr>

* Nhìn qua các comment, ta sẽ thấy có các comment chứa emoji như hình dưới đây:<br>
  ![](./images/05.png)

* Đây là "vốn quý" góp phần làm tăng sức mạnh cho model, nếu ta thực hiện bước loại bỏ các kí tự đặc biệt trước khi ta tách các emoji ra, thì ta đã vô tình xóa luôn các emoji này, vì các emoji thực chất được xây dựng dựa trên các kí tự đặc biệt.
* Như hình trên, rõ ràng ta thấy được emoji góp phần ta hiểu được một comment là positive hay negative.
* Ta sẽ sử dụng một gói của python là `emojis`:
  ```shell
  pip3 install emojis
  ```
  gói này sẽ giúp ta tách các emoji ra khỏi bình luận.
* Ta sẽ chứa toàn bộ emoji của một comment qua cột tương ứng là `emoji`

In [22]:
reviews['emoji'] = reviews['raw_comment'].apply(lambda cmt: Processor.extractEmoji(cmt))

reviews.head()

,raw_comment,label,normalize_comment,emoji
0,"Mình mua size L. Cổ tay siêu bé, như siz...",0,"mình mua size l. cổ tay siêu bé, như siz...",
1,"Size S M L XL\n\nForm áo cực kì dễ mang,...",1,"size s m l xl\n\nform áo cực kì dễ mang,...",
2,Áo khá đẹp vừa với dáng giao hàng cư...,1,áo khá đẹp vừa với dáng giao hàng cư...,
3,Đẹp rất hài lòng okokokokokokokokokokokok...,1,đẹp rất hài lòng okokokokokokokokokokokok...,
4,"Đẹp, ôm dáng, mặc đẹp lắm mà form nho...",1,"đẹp, ôm dáng, mặc đẹp lắm mà form nho...",


In [23]:
",".join(reviews['emoji'])[:200]

',,,,,,,,,,,,,,,,💕,,,,🥰,,,,,✋,,,,,,,,,,,😘,,,,,,,,👍,,,,,,🥰,,,,,😍,,,,,,,,👍,,,,,,🥰,,,,,,,,,,,,,,,👍,,,👍,,,,❤️,,,,,,,,,,,,,,,,,,,,,,,,,😑,,,,,,,\U0001f972,,,,,,,,,,,,🤡 🥰 😘,,,,,,🐸 🌾,,,😂,,,🤬,,,,,,♥️ 🥺,,,,,,,,🙈,,,,,,,,,'

<hr>

* Tiếp theo, ta sẽ loại bỏ dấu câu, kí tự đặc biệt

In [24]:
reviews['normalize_comment'] = reviews['raw_comment'].apply(lambda cmt: Processor.removeSpecialLetters(cmt.lower()))

reviews.head()

,raw_comment,label,normalize_comment,emoji
0,"Mình mua size L. Cổ tay siêu bé, như siz...",0,mình mua size l cổ tay siêu bé như size ...,
1,"Size S M L XL\n\nForm áo cực kì dễ mang,...",1,size s m l xl form áo cực kì dễ mang thi...,
2,Áo khá đẹp vừa với dáng giao hàng cư...,1,áo khá đẹp vừa với dáng giao hàng cư...,
3,Đẹp rất hài lòng okokokokokokokokokokokok...,1,đẹp rất hài lòng okokokokokokokokokokokok...,
4,"Đẹp, ôm dáng, mặc đẹp lắm mà form nho...",1,đẹp ôm dáng mặc đẹp lắm mà form nhỏ ...,
